<a href="https://colab.research.google.com/github/andreldeod/IDCE-302-Python-Programming/blob/main/AndreSubGroup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Cluster steps to run Prithvi-100M for fairy-ring mapping of cranberry bogs.

André

##Introduction

My objective as this sub-group was to pre-process data and do an initial fine-tuning and prediction run of the Privthis-100M model on mapping fairy-ring pathogens in cranberry fields of New Jersey. My goal was to provide ready-to use input data and established workflows to facilitate further fine-tuning by the other subgroups. For the initial run I tried to stay as true to the parameters used in the fine-tuning configuration shared and explained by Sam Khallaghi. Sam, Professor Lyndon Estes and our TA, Rahebeh Abedi, provided an immense amount of help and guidance in this project. We also adapted code written by Hanxi Li for creating our image and label chips. Thank you!

##Methods
###Data
* We used an orthomosaic drone image of a cranberry farm in NJ from July 2015.
* Labels were created by all members of the group.


###Model
* We used the [Prithvi-100M](https://huggingface.co/ibm-nasa-geospatial/Prithvi-100M/blob/main/README.md) foundation model.
* The configuration file was an adaptation of Sam Khallaghi's fine-tuning configuration for mapping aquaculture.

###Workflow
1. [Set-up cluster environments.](https://github.com/kordi1372/cranberry/blob/main/notebooks/cluster_steps.ipynb) (Notebook contains steps for referencing coming from hls-foundation-os [repo](https://github.com/NASA-IMPACT/hls-foundation-os/tree/main))
2. [Calculate class weights.](https://github.com/kordi1372/cranberry/blob/main/notebooks/class_weights.ipynb)
3. [Chip input image and labels](https://github.com/kordi1372/cranberry/blob/main/notebooks/chipping.ipynb)
    * See [ImageProcessing folder](https://github.com/kordi1372/cranberry/tree/main/cranberry/imageProcessing) for modules used in the chipping notebook.
4. Use [configuration file](https://github.com/kordi1372/cranberry/blob/main/cranberry/cranberry_Andre_configuration.py) for fine-tuning the model.

```
mim train mmsegmentation /home/airg/adomingues/cranberry/models/cranberry_Andre.py
```
5. Use best saved epoch from fine-tuning to run model predictions on images to create a binary classification showing presence and absence of fairy rings.
```
python /home/airg/adomingues/cranberry/models/hls-foundation-os-main/model_inference.py -config /home/airg/adomingues/cranberry/models/cranberry_Andre_finetuned.py -ckpt /home/airg/adomingues/cranberry/outputs/run1_May4/best_mIoU_epoch_15.pth  -input /home/airg/adomingues/cranberry/data/chips_for_predictions/ -output /home/airg/adomingues/cranberry/outputs/run2_May6/ -input_type tif -bands 0 1 2 3 4 5
```
6. [Merge prediction chips](https://github.com/kordi1372/cranberry/blob/main/notebooks/merging_chips.ipynb) into a single binary classification raster.





Project Working Directory:
```
cranberry -> data       -> prediction_chips
                        -> labels
                        -> images
                        -> val_images.txt
                        -> train_images.txt
                                    
          -> models     -> hls-foundation-os-main (from git clone)
                        -> mmsegmentation (from git clone)
                        -> cranberry_Andre.py
                        -> Prithvi_100M.pt
          
          -> outputs    -> run_May7

```





##Results/analysis

The first run of our fine-tune model was a bit unstable. Our best epoch was early on, at epoch 15 out of 80. The fairy ring class had an accuracy of 63.17, while non-fairy ring class had an accuracy of 82.72.

The model was run with a fully convolutional network (FCN) encoder head. Cross entropy loss was used for the loss function. The class weights were calculated with the balance cross entropy loss function. We used small batches of 4 and a low running rate of 1.5e-05. We used AdamW for an adaptive optimizer.

The prediction [image](https://github.com/kordi1372/cranberry/blob/main/cranberry/outputs/run_1_prediction_andre.png) from the first run, shows that the model did relatively well at identifying the fairy rings that were labeled. In some cases, like in the bottom left field, the model classified faint fairy rings that were missed by the labeler. There was more commission than commission. The model falsely classified forest canopy areas with lots of shadows as fairy rings in the north east corner of the image. The model did well at distinguishing the flooded cranberry bog and paved roads and fallow fields from fairy-rings. The model likely overclassified the fairy rings present in the bottom right. It is difficult to tell from visual inspection if the darker surfaces in this field are fairy ring related or not, but they lack the circular pattern.

For future runs of Prithvi-100M on fairy ring cranberry mapping, it would be interesting to experiment with different decoder heads and loss functions to improve the models generalizability. The predictions should also be applied different ortho images.



![Run 1 prediction](cranberry/outputs/run_1_prediction_andre.png "Run 1 prediction")

Fairy ring classification. Presence in white, absence in black. Green outlines are the hand made labels. This prediction is from our first run of the fine-tuned model.

## Testing metrics from fine-tunning the model
---
Epoch 5:
```
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
| not_fairyring | 64.41 | 88.71 |
|   fairyring   | 46.88 | 53.42 |
+---------------+-------+-------+
    +-------+-------+-------+
    |  aAcc |  mIoU |  mAcc |
    +-------+-------+-------+
    | 72.92 | 55.65 | 71.06 |
    +-------+-------+-------+
```
---
Epoch 15  (**BEST EPOCH**) :
```
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
| not_fairyring | 63.72 | 82.72 |
|   fairyring   | 52.06 | 63.17 |
+---------------+-------+-------+
    +-------+-------+-------+
    |  aAcc |  mIoU |  mAcc |
    +-------+-------+-------+
    | 73.97 | 57.89 | 72.95 |
    +-------+-------+-------+
```
---
Epoch 30:
```
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
| not_fairyring | 61.89 | 84.55 |
|   fairyring   | 46.01 | 54.79 |
+---------------+-------+-------+
    +-------+-------+-------+
    |  aAcc |  mIoU |  mAcc |
    +-------+-------+-------+
    | 71.23 | 53.95 | 69.67 |
    +-------+-------+-------+
```
---
Epoch 45:
```
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
| not_fairyring | 60.81 | 84.58 |
|   fairyring   | 43.44 | 51.71 |
+---------------+-------+-------+
    +-------+-------+-------+
    |  aAcc |  mIoU |  mAcc |
    +-------+-------+-------+
    | 69.88 | 52.13 | 68.15 |
    +-------+-------+-------+
```
---
Epoch 60:
```
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
| not_fairyring | 60.37 | 82.56 |
|   fairyring   | 44.93 |  54.6 |
+---------------+-------+-------+
    +-------+-------+-------+
    |  aAcc |  mIoU |  mAcc |
    +-------+-------+-------+
    | 70.05 | 52.65 | 68.58 |
    +-------+-------+-------+
```
---
Epoch 75:
```
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
| not_fairyring | 59.78 | 80.98 |
|   fairyring   | 45.51 |  56.2 |
+---------------+-------+-------+
    +-------+-------+-------+
    |  aAcc |  mIoU |  mAcc |
    +-------+-------+-------+
    | 69.89 | 52.65 | 68.59 |
    +-------+-------+-------+
```
---
Epoch 80:
```
+---------------+-------+-------+
|     Class     |  IoU  |  Acc  |
+---------------+-------+-------+
| not_fairyring | 59.77 | 81.24 |
|   fairyring   | 45.17 | 55.63 |
+---------------+-------+-------+
    +-------+-------+-------+
    |  aAcc |  mIoU |  mAcc |
    +-------+-------+-------+
    | 69.78 | 52.47 | 68.44 |
    +-------+-------+-------+
```